In [7]:
#image = cv2.imread("image1.jpg")
'''def find_rec(cnt):
    rects = [cv2.boundingRect(ctr) for ctr in cnt]
    area=[]
    for i in rects:
        area.append(i[2]*i[3])
    
    return area.index(max(area)),rects

image = cv2.imread("sudoku.jpeg")

contours,_ = im_process(image)

#cv2.drawContours( image, contours, -1, (128,255,255),3, cv2.LINE_AA)

ind,rects = find_rec(contours)

rect = rects[ind]

img = image[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]]

cv2.imshow("sudoku", img)
cv2.waitKey()
cv2.destroyWindow("sudoku")'''

'def find_rec(cnt):\n    rects = [cv2.boundingRect(ctr) for ctr in cnt]\n    area=[]\n    for i in rects:\n        area.append(i[2]*i[3])\n    \n    return area.index(max(area)),rects\n\nimage = cv2.imread("sudoku.jpeg")\n\ncontours,_ = im_process(image)\n\n#cv2.drawContours( image, contours, -1, (128,255,255),3, cv2.LINE_AA)\n\nind,rects = find_rec(contours)\n\nrect = rects[ind]\n\nimg = image[rect[1]:rect[1]+rect[3],rect[0]:rect[0]+rect[2]]\n\ncv2.imshow("sudoku", img)\ncv2.waitKey()\ncv2.destroyWindow("sudoku")'

In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math 
#For scikit learn hog
import joblib
from skimage.feature import hog
#For tensorflow cnn model
import tensorflow as tf
from tensorflow import keras

In [10]:
def im_process(image):
    gray_image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    #gray_image = cv2.GaussianBlur(gray_image,(5, 5), 0)
    #gray_image = cv2.blur(gray_image,(5,5))
    #gray_image = cv2.medianBlur(gray_image,5)
    gray_image = cv2.bilateralFilter(gray_image,9,75,75)
    
    thresh = cv2.adaptiveThreshold(gray_image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,9,2)

    contours ,_ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours,thresh

In [11]:
def find_rect(cnt):
    rects = [cv2.boundingRect(ctr) for ctr in cnt]
    area=[]
    for i in rects:
        area.append(i[2]*i[3])
    
    return cnt[area.index(max(area))],rects

In [12]:
def rhom(thresh):
    lst = []
    for i in range(10,int(thresh.shape[0]/2)):
        lst.append(sum(thresh[i,int(thresh.shape[1]/2)-i:int(thresh.shape[1]/2)+i+1]))
    j=0
    for i in range(int(thresh.shape[0]/2),thresh.shape[0]-10):
        j+=1
        lst.append(sum(thresh[i,i+1-int(thresh.shape[1]/2):int(thresh.shape[1])-j]))
    return sum(lst)

In [13]:
new_model = tf.keras.models.load_model('tf_model.h5')

In [50]:
img = cv2.imread("sudoku.jpeg")

contours,thresh = im_process(img)
"""cv2.imshow("sudoku1",thresh)
cv2.waitKey(0)
cv2.destroyWindow("sudoku1")"""
cnt,_ = find_rect(contours)

rect = cv2.minAreaRect(cnt)

box = cv2.boxPoints(rect)

width = int(rect[1][0])
height = int(rect[1][1])

dst_pts = np.array([[0, height-1],[0, 0],[width-1, 0],[width-1, height-1]], dtype="float32")

img = cv2.warpPerspective(img,cv2.getPerspectiveTransform(box, dst_pts), (width, height))
#img = img[10:int(img.shape[0])-10,10:int(img.shape[1])-10]
row = img.shape[0]/9
col = img.shape[1]/9

for i in range(1):
    for j in range(0,9):
        image = img[int(row*i):int(row*(i+1)),int(col*j):int(col*(j+1))]
        cnt ,thresh = im_process(image)
        sm = rhom(thresh)
        if sm>50000:
            roi = cv2.resize(thresh, (28,28), interpolation=cv2.INTER_NEAREST).reshape(28,28,1)
            #roi2 = roi.reshape(28, 28, 1)
            sample = []
            sample.append(roi)
            sample = np.array(sample)
            lst = new_model.predict(sample)
            classes = np.argmax(lst, axis = 1)
            print(classes)
            cv2.imshow("sudoku",roi)
            cv2.waitKey(0)
            cv2.destroyWindow("sudoku")

'''cv2.imshow("sudoku1",img)
cv2.waitKey(0)
cv2.destroyWindow("sudoku1")'''


[6]
[0]


'cv2.imshow("sudoku1",img)\ncv2.waitKey(0)\ncv2.destroyWindow("sudoku1")'